In [1]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir("drive/MyDrive/Bootcamp_Henry/pro_final")
print(os.getcwd())

Mounted at /content/drive
/content/drive/MyDrive/Bootcamp_Henry/pro_final


# **Objetivo del Cuaderno Jupyter**
- A partir de los archivos recibidos del exterior que contienen los datos de la plataforma `YELP` y apoyandonos en el `Maestro de Categorías` realizado en el equipo, realizar el ETL local de las tablas de yelp que se muestran en la imagen.
<img src="0_TABLAS_YELP.png">
<br>
<br>
# **NOTA:**<br>
Sí desea ver el proceso que van sufriendo los datos desde que llegan en los archivos Iniciales hasta el Datawarehouse, es recomendable observar el archivo `1_etl_y_filtrado_yelp.ipnb` donde se va mas paso a paso y se van ejecutando celdas y se puede apreciar mas el detalle.  Acá sólo es una base de funciones de las que nuestro  nuestro equipo de ingeniería se guiará, modificará y adaptará (origenes, destinos, sintáxis en caso de utilizar otra tecnología, etc) para cumplir dicho cometido en la Nube


Importando Librerias

In [ ]:
import pandas as pd
import datetime
import requests
import time
import gc

## Función que retorna los 10 primeros caracteres de una cadena

In [ ]:
def devuelve_solo_fecha(campo:str) -> str:
  """
  devuelve_solo_fecha: Devuelve los 10 primeros caracteres de una cadena  
  Args:
      campo (str): cadena recibida

  Returns:
      str: 10 primeros caracteres de la cadena
  """
  return campo.strip()[:10]

## Función que retorna los Horarios de los dias

In [ ]:
def retornar_horario(data:dict|str, dia:str) -> str:
  """
  retornar_horario: Recibe el diccionario con los horario y el dia en question y
                    devuelve el horario de ese dia
  Args:
      data (dict | str): Diccionario de Horarios | string en caso "No Disponible"
      dia (str): Dia del que se quiere el horario

  Returns:
      str: Horario del dia preguntado
  """
  if data == 'No Disponible':
    return 'No Disponible'
  else:
    if dia in data.keys():
      return data[dia]
    else:
      return "No Disponible"

## Función que obtiene los estados según el codigo Postal

In [ ]:
def get_states_by_postal_code(lista_codigos_postales:list) -> list:
  """
  get_states_by_postal_code: Consulta Una Api Gratuita de EEUU para obtener los estados
                             en base al codigo postal

  Args:
      lista_codigos_postales (list): Lista de los codigos postales de los registros de la
                                     tabla business de YELP

  Returns:
      list: Lista de diccionarios con los codigos postales y sus respectivos estados
  """
  estados_obtenidos_api = []
  for i in range(len(lista_codigos_postales)):
    codigo_postal = lista_codigos_postales[i]
    diccionario = {}
    try:
      codigo_numero = int(lista_codigos_postales[i])
      url = f"http://api.zippopotam.us/us/{codigo_numero}"
      response = requests.get(url)
      if response.status_code == 200:
        data = response.json()
        estado = data['places'][0]['state abbreviation']
        diccionario['codigo_postal'] = codigo_postal
        diccionario['state'] = estado
      else:
        diccionario['codigo_postal'] = codigo_postal
        diccionario['state'] = 'BORRAR REGISTRO'
    except:
      diccionario['codigo_postal'] = codigo_postal
      diccionario['state'] = 'BORRAR REGISTRO'

      estados_obtenidos_api.append(diccionario)
      time.sleep(2)

  return estados_obtenidos_api

## Funcion que crea `y_business` y `y_attributes_busines`

In [ ]:
def create_business_attributes(business:pd.Dataframe, business_list:list):
  """
  create_business_attributes: Crea las tablas y_business y y_attributes_busines

  Args:
      business (pd.Dataframe): Contiene la data de business como viene originalmente
      business_list (list):    Listado de business_id del sector en estuio
  """
  business = business.iloc[:,0:14]
  #Obtengo la lista de los codigos postales de los negocios
  lista_codigos_postales = business['postal_code'].unique().tolist()
  #Llamo a la funcion que busca averigua los estados con la Api de Codigos Postales en EEUU
  estados_obtenidos_api = get_states_by_postal_code(lista_codigos_postales)

  codigos_postales_api = pd.DataFrame(estados_obtenidos_api)
  # Ciclo for para asignar los estados según el codigo postal
  for i in range(business.shape[0]):
    codigo_postal = business.iloc[i,5]
    # Filtrando el dataframe que se obtuvo de la Api de Codigos Postales
    mask = codigos_postales_api['codigo_postal'] == codigo_postal
    business.iloc[i,4] = codigos_postales_api.loc[mask]['state'].values[0]

  # Borrando los registros de los Negocios que no petenecen a EEUU
  mask = business['state'] != 'BORRAR REGISTRO'
  business = business.loc[mask]

  # Filtrando Según Maestro de Categorias
  mask = business['business_id'].isin(business_list)
  business = business.loc[mask]

  # Reemplazando los Nulos por `No Disponible`
  business.loc[:, 'attributes'] = business['attributes'].fillna('No Disponible')
  business.loc[:, 'hours'] = business['hours'].fillna('No Disponible')

  # Creando la tabla `y_attributes_business`
  attributes = business[['business_id', 'attributes']]

  attributes = attributes[attributes['attributes'] != 'No Disponible']

  # creando las columnas `monday`, `tuesday`, `wednesday`, `thursday`, `friday`, `saturday`, `sunday`
  business['monday'] = business['hours'].apply(lambda x: retornar_horario(x, dia='Monday'))
  business['tuesday'] = business['hours'].apply(lambda x: retornar_horario(x, dia='Tuesday'))
  business['wednesday'] = business['hours'].apply(lambda x: retornar_horario(x, dia='Wednesday'))
  business['thursday'] = business['hours'].apply(lambda x: retornar_horario(x, dia='Thursday'))
  business['friday'] = business['hours'].apply(lambda x: retornar_horario(x, dia='Friday'))
  business['saturday'] = business['hours'].apply(lambda x: retornar_horario(x, dia='Saturday'))
  business['sunday'] = business['hours'].apply(lambda x: retornar_horario(x, dia='Sunday'))

  business = business.drop(columns=['attributes', 'categories', 'hours'])

  # Guardando la tabla `y_business`
  business.to_parquet('datasets/yelp/filtrados_segun_maestro_cat/y_business.parquet', index=False)

  # Guardando la tabla `y_attributes_business`
  attributes.to_parquet('datasets/yelp/filtrados_segun_maestro_cat/y_attributes_business.parquet', index=False)

## Funcion que crea `y_tip` y `y_checkin`

In [ ]:
def create_tip_checkin(tip:pd.Dataframe, checkin:pd.Dataframe, business_list:list):
  """
  create_tip_checkin : Crea las tablas y_tip y y_checkin

  Args:
      tip (pd.Dataframe): Contiene la data de tip como viene originalmente
      checkin (pd.Dataframe): Contiene la data de checkin como viene originalmente
      business_list (list): Listado de business_id del sector en estuio
  """
  #filtro segun los id que se obtuvieron del maestro de categorias
  mask = tip['business_id'].isin(business_list)
  tip = tip.loc[mask]
  tip['date_only'] = tip['date'].dt.date
  tip = tip.drop(columns=['date'])
  tip = tip.rename(columns={'date_only': 'date'})

  checkin['lista_date'] = checkin['date'].str.split(',')
  checkin = checkin.drop(columns=['date'])
  checkin = checkin.explode('lista_date')
  checkin = checkin.rename(columns={'lista_date': 'date'})
  checkin['only_date'] = checkin['date'].apply(devuelve_solo_fecha)
  checkin['only_date'] = pd.to_datetime(checkin['only_date'])
  checkin = checkin.drop(columns=['date'])
  checkin = checkin.rename(columns={'only_date': 'date'})

  tip.to_parquet('datasets/yelp/filtrados_segun_maestro_cat/y_tip.parquet', index=False)
  checkin.to_parquet('datasets/yelp/filtrados_segun_maestro_cat/y_checkin.parquet', index=False)

## Funcion que crea `y_review`

In [ ]:
def create_review(reviews:pd.Dataframe, business_list:list) -> list:
  """
  create_review: Crea la tabla y_review

  Args:
      reviews (pd.Dataframe): Contiene la data de review como viene originalmente
      business_list (list): Listado de business_id del sector en estudio

  Returns:
      list: Lista de los id de los usuarios involucrados en los reviews del sector de estudio
  """
  mask = reviews['business_id'].isin(business_list)
  reviews = reviews.loc[mask]

  reviews['only_date'] = reviews['date'].apply(devuelve_solo_fecha)
  reviews['only_date'] = pd.to_datetime(reviews['only_date'])

  reviews = reviews.drop('date')
  reviews = reviews.rename({'only_date':'date'})

  # ESCOGIENDO LAS RESEÑAS A PARTIR DEL 2021
  mask = reviews['date'] > datetime.strptime('2020-12-31', '%Y-%m-%d')
  reviews = reviews.loc[mask]
  # Guardando la Tabla
  reviews.to_parquet('datasets/yelp/filtrados_segun_maestro_cat/y_review.parquet', index=False)
  # Dado que filtramos desde 2021 filtro obtengo lista de usuarios
  lista_usuarios_a_filtrar = reviews['user_id'].unique().to_list()

  return lista_usuarios_a_filtrar

## Funcion que crea `y_user`, `y_elite_user`, `y_friends_user`

In [ ]:
def create_user_elite_friends(user:pd.Dataframe, user_list:list):
  """
  create_user_elite_friends: Crea las tablas `y_user`, `y_elite`, `y_friends_users`
  Args:
      user (pd.Dataframe): Contiene la data de user como viene originalmente
      user_list (list): Listado de user_id que hicieron reseñas en el sector en estudio
  """
  mask = user['user_id'].isin(user_list)
  user = user.loc[mask]
  user = user.unique()
  user['date'] = user['yelping_since'].apply(devuelve_solo_fecha)
  user['only_date'] = pd.to_datetime(user['date'])
  user = user.drop(columns=['yelping_since'])
  user = user.rename(columns={'date': 'yelping_since'})

  # Se crean tablas `y_fiends_user` y `y_elite_user`
  friends_user = user[['user_id', 'friends']]
  elite_user = user[['user_id', 'elite']]

  # Se borran las columnas de la tabla `y_user`
  user = user.drop(columns=['friends', elite])

  # Se trabaja en la Tabla `y_friends_user`
  friends_user.columns = ['user_id', 'friend_id']
  mask = friends_user['friend_id'] == 'None'
  friends_user = friends_user.loc[~mask]
  friends_user['lista_friends'] = friends_user['friend_id'].apply(lambda x: x.split(','))
  friends_user = friends_user.drop(columns=['friend_id'])
  friends_user = friends_user.explode('lista_friends')
  friends_user.columns = ['user_id', 'friend_id']

  # Se trabaja en la Tabla `y_elite_user`
  elite_user.columns = ['user_id', 'elite']
  mask = elite_user['elite'] == ''
  elite_user = elite_user.loc[~mask]
  elite_user['year'] = elite_user['elite'].apply(lambda x: x.split(','))
  elite_user = elite_user.drop(columns=['elite'])
  elite_user = elite_user.explode('year')
  elite_user = elite_user.unique()
  elite_user.loc[elite_user['year'] == '20', 'year'] = '2020'
  elite_user['year'] = elite_user['year'].astype(int)

  # Se guardan las 3 tablas
  user.to_parquet('datasets/yelp/filtrados_segun_maestro_cat/y_user.parquet', index=False)
  friends_user.to_parquet('datasets/yelp/filtrados_segun_maestro_cat/y_friends_user.parquet', index=False)
  elite_user.to_parquet('datasets/yelp/filtrados_segun_maestro_cat/y_elite_user.parquet', index=False)

## Importando el archivo `business.pkl` tal como viene Original

In [ ]:
business = pd.read_pickle('datasets/yelp/archivos_originales/business.pkl')

#### Importacion del archivo que en el EDR se llama `y_categories_business` como quedó despues de haber hecho el Maestro de Categorias

In [ ]:
categorias_yelp = pd.read_parquet('datasets/categorias/y_categories_business.parquet')

## Importando el archivo `tip.json` tal como viene Original

In [ ]:
tip = pd.read_json('datasets/yelp/archivos_originales/tip.json', lines=True)

## Importando el archivo `checkin.json` tal como viene Original

In [ ]:
checkin = pd.read_json('datasets/yelp/archivos_originales/checkin.json', lines=True)

## Importando el archivo `review.json` tal como viene Original

In [ ]:
reviews = pd.read_ndjson("datasets/yelp/review.json", lines=True)

## Importando el archivo `user.json` tal como viene Original

In [ ]:
user = pd.read_parquet("datasets/yelp/archivos_originales/user.parquet")

## Obteniendo la lista de Negocios según el Maestro de Categorias

In [ ]:
# Filtro en Categorias por alimentos y bebidas y obtengo una lista de los negocios
mask = categorias_yelp['category_general'] == 'Alimentos y Bebidas'
lista_business_alimentos_bebidas = categorias_yelp.loc[mask]['business_id'].unique().tolist()

## Llamado a la Función que crea `y_business` y `y_attributes_business`

In [ ]:
create_business_attributes(business=business, business_list=lista_business_alimentos_bebidas)

## Llamado a la Función que crea `y_tip` y `y_checkin`

In [ ]:
create_tip_checkin(tip=tip, checkin=checkin, business_list=lista_business_alimentos_bebidas)

## Llamado a la Función que crea `y_review`

In [ ]:
lista_usuarios_a_filtrar = create_review(reviews=reviews, business_list=lista_business_alimentos_bebidas)

## Llamado a la Función que crea `y_user`, `y_elite_user` y `y_friends_user`

In [ ]:
create_user_elite_friends(user=user, user_list=lista_usuarios_a_filtrar)